In [11]:
%matplotlib inline

# Load the modules
import pickle
import math

import numpy as np
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt

# Reload the data
pickle_file = 'notMNIST.pickle'
with open(pickle_file, 'rb') as f:
  pickle_data = pickle.load(f)
  train_features = pickle_data['train_dataset']
  train_labels = pickle_data['train_labels']
  valid_features = pickle_data['valid_dataset']
  valid_labels = pickle_data['valid_labels']
  test_features = pickle_data['test_dataset']
  test_labels = pickle_data['test_labels']
  del pickle_data  # Free up memory


print('Data and modules loaded.')

Data and modules loaded.


In [12]:
# Data has been normalized 
total_samples = len(train_features)

# Parameters
learning_rate = 0.008
num_output = 10
epochs = 100
batch_size = 500
steps_per_epoch = int(np.ceil(total_samples / batch_size))

In [13]:
# Cleaning Data
#valid_labels, test_labels = np.array(valid_labels, np.int64), np.array(test_labels, np.int64)
train_data = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
val_features, test_features = np.array(valid_features, np.float32), np.array(test_features, np.float32)

# Data has been normalized

In [14]:
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [15]:
features = train_features.shape[1]

In [16]:
W = tf.Variable(tf.random.normal([features, num_output], name="weights1"))
B = tf.Variable(tf.zeros([num_output], name="bias1"))

optimizer = tf.optimizers.SGD(learning_rate)

# Model
class MyModel(tf.Module):
    def __call__(self, X):
        return tf.nn.softmax(tf.add(tf.matmul(X, W), B))

def cross_entropy(y_pred, y_true):
    # It has been one-hot encoded before storing as pickle
    #y_true = tf.one_hot(y_true, depth=num_output) 
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred), 1))

def accuracy(y_pred, y_true):
    if len(y_true.shape) > 1 and y_true.shape[1] > 1:
        y_true = tf.argmax(y_true, axis=1)
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def run_optimizer(X, Y):
    with tf.GradientTape() as g:
        logit = model(X)
        loss = cross_entropy(logit, Y)

    gradients = g.gradient(loss, [W, B])
    optimizer.apply_gradients(zip(gradients, [W, B]))
    return None

def batch_data(X, Y, batch_size):
    output_data = []
    sample_size = len(X)
    for step in range(0, sample_size, batch_size):
        start = batch_size * step
        end = batch_size + start
        batch_X = X[start:end]
        batch_Y = Y[start:end]
        yield batch_X, batch_Y

model=MyModel()

checkpoint = tf.train.Checkpoint(model = model)
checkpoint.save("./checkpoints/mymodel")
manager = tf.train.CheckpointManager(checkpoint, "./checkpoints", max_to_keep=3)

In [17]:
# Training
for epoch in range(1, epochs + 1):
    for step, (batch_X, batch_Y) in enumerate(train_data.take(steps_per_epoch), 1):
        run_optimizer(batch_X, batch_Y)

    val_pred = model(valid_features)
    val_loss = cross_entropy(val_pred, valid_labels)
    val_acc = accuracy(val_pred, valid_labels)
    train_pred = model(batch_X)
    train_loss = cross_entropy(train_pred, batch_Y)
    train_acc = accuracy(train_pred, batch_Y)
    manager.save()
    
    print(f"Epoch: {epoch}, train_loss {train_loss}, train_accuracy {train_acc}, val_loss {val_loss}, val_accuracy {val_acc}")

Epoch: 1, train_loss 12.338140487670898, train_accuracy 0.21799999475479126, val_loss 12.678192138671875, val_accuracy 0.1891999989748001
Epoch: 2, train_loss 11.177257537841797, train_accuracy 0.25200000405311584, val_loss 11.251325607299805, val_accuracy 0.23520000278949738
Epoch: 3, train_loss 9.799777030944824, train_accuracy 0.2619999945163727, val_loss 9.643571853637695, val_accuracy 0.26306667923927307
Epoch: 4, train_loss 8.577306747436523, train_accuracy 0.3019999861717224, val_loss 8.573391914367676, val_accuracy 0.2939999997615814
Epoch: 5, train_loss 7.493995666503906, train_accuracy 0.33399999141693115, val_loss 7.161098003387451, val_accuracy 0.3202666640281677
Epoch: 6, train_loss 6.142388343811035, train_accuracy 0.38999998569488525, val_loss 6.284374713897705, val_accuracy 0.3898666799068451
Epoch: 7, train_loss 6.32255220413208, train_accuracy 0.45399999618530273, val_loss 5.756913661956787, val_accuracy 0.4413333237171173
Epoch: 8, train_loss 5.473896503448486, train

2025-11-03 09:00:38.776289: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch: 28, train_loss 4.077287197113037, train_accuracy 0.6579999923706055, val_loss 4.071807384490967, val_accuracy 0.6446666717529297
Epoch: 29, train_loss 3.767143726348877, train_accuracy 0.6740000247955322, val_loss 4.0494866371154785, val_accuracy 0.646133303642273
Epoch: 30, train_loss 3.626960515975952, train_accuracy 0.6579999923706055, val_loss 4.027705669403076, val_accuracy 0.6478666663169861
Epoch: 31, train_loss 4.343687057495117, train_accuracy 0.6380000114440918, val_loss 4.0054731369018555, val_accuracy 0.6489333510398865
Epoch: 32, train_loss 3.6914846897125244, train_accuracy 0.6520000100135803, val_loss 3.983588218688965, val_accuracy 0.6510666608810425
Epoch: 33, train_loss 3.7999229431152344, train_accuracy 0.6480000019073486, val_loss 3.9614980220794678, val_accuracy 0.653333306312561
Epoch: 34, train_loss 3.828500270843506, train_accuracy 0.6700000166893005, val_loss 3.9392876625061035, val_accuracy 0.6553333401679993
Epoch: 35, train_loss 3.813655138015747, tra

In [18]:
test_pred = model(test_features)
test_loss = cross_entropy(test_pred, test_labels)
test_acc = accuracy(test_pred, test_labels)
print(f"Test_loss {test_loss}, Test_accuracy {test_acc}")

Test_loss 1.0521560907363892, Test_accuracy 0.8331999778747559
